In [18]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *
import random


# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 10
tiles_vert = 7

antal_celler  = 7
start_energi = 10

# List to contain the cell objects
cells = []

# Create empty world, to contain references to cells
verd = []
for x in range(tiles_hori):
    row = []
    for y in range(tiles_vert):
        row.append(None)
    verd.append(row)

# Populate world
for i in range(0, antal_celler):
    x = random.randint(0, tiles_hori - 1)
    y = random.randint(0, tiles_vert - 1)
    while verd[x][y] != None and verd:
        x = random.randint(0, tiles_hori - 1)
        y = random.randint(0, tiles_vert - 1)
    new_cell = Celle(x, y, start_energi, [])
    cells.append(new_cell)
    verd[x][y] = len(cells) - 1

# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)

def draw_world():
    # Draw grid
    canvas.fill_style = "gray"
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)


with hold_canvas():
    draw_world()
    for cell in cells:
        cell.tegn(canvas, tile_size, rect_size)

Canvas(height=105, width=150)

TypeError: Celle.tegn() missing 3 required positional arguments: 'canvas', 'tile_size', and 'rect_size'